In [1]:
import pymongo
import numpy as np
import pandas as pd
import datetime,time

In [5]:
# Connect with mongodb

try:
    with open("credentials.txt", 'r', encoding='utf-8') as f:
        [name,password,url,dbname]=f.read().splitlines()
        conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
    print ("Connected successfully!!!")
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e) 
    

    
db = conn['tweempact']
collection = db['tweets']

Connected successfully!!!


In [6]:
db.collection_names()

C:\Users\Albert Prat\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['system.indexes', 'tweets_noRT', 'users_noRT', 'tweets', 'tweet_users']

In [7]:
def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """
    
    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']
    return df

In [19]:
df1 = read_mongo(db, "tweets")
df2 = read_mongo(db, "tweets_noRT")

In [20]:
print(len(df1), len(df2))

10490 19119


In [13]:
 def author_dataset_conversion(df):   
    import time
    import numpy as np
    import pandas as pd

    cleaned = df[["id", "created_at","entities", "extended_entities","favorite_count","retweet_count","source","text", "truncated","user","quoted_status_id"]]
    ids = [user["id"] for user in cleaned["user"]]
    cleaned["id_num"] = ids

    #preparing output variables
    output = pd.DataFrame()
    users = []
    mean_RT_last10,mean_FC_last10 = [],[]
    sd_RT,sd_FC  = [],[]
    y1,y2=[],[]

    for _id in np.unique(ids):
        subset = cleaned[cleaned["id_num"]==_id].reset_index(drop=True)

        #filtering tweets from all tweets and RT
        l = list(subset["text"])
        filtr = ["RT" not in i for i in l]
        subset = subset[pd.Series(filtr)]
        if len(subset)>=11:
            reindexed = subset["created_at"].reset_index(drop=True)
            #convert tweet date into python date
            subset["created_at_pydate"] = [time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(reindexed[i],'%a %b %d %H:%M:%S +0000 %Y'))for i in range(0,len(reindexed))]
            subset = subset.sort_values(by="created_at_pydate").reset_index(drop=True)

            last10_tweets = subset[-11:-1]
            last_tweet = subset[-1:]
            #RT parameters last 10 tweets
            mean_RT_last10.append(np.mean(last10_tweets["retweet_count"]))
            sd_RT.append(np.std(last10_tweets["retweet_count"]))
            #FC parameters last 10 tweets
            mean_FC_last10.append(np.mean(last10_tweets["favorite_count"]))
            sd_FC.append(np.std(last10_tweets["favorite_count"]))
            #take json user
            j_user = subset[-1:]["user"].reset_index(drop=True)[0]
            users.append(j_user)
            #parameters of last tweet
            y1.append(np.mean(last_tweet["favorite_count"]))
            y2.append(np.mean(last_tweet["retweet_count"]))
    output["j_user"]=users
    output["RT_l10"]=mean_RT_last10
    output["sd_RT"]=sd_RT
    output["FC_l10"]=mean_FC_last10
    output["sd_FC"]=sd_FC
    output["FC"]=y1
    output["RT"]=y2
    return output

In [22]:
output1 = author_dataset_conversion(df1)
output2 = author_dataset_conversion(df2)

C:\Users\Albert Prat\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [26]:
print(len(output1),len(output2))

186 1406


In [33]:
output = pd.concat([output1, output2], axis = 0)

In [35]:
pd.DataFrame.to_csv(output, "C:/Users/Albert Prat/Documents/tweempact/datasets/author.csv")

In [72]:
def text_dataset(df):
    import re
    import pandas as pd
    l = list(df["text"])
    filtr = ["RT" not in i for i in l]
    df = df[filtr].reset_index(drop=True)
    
    Tweet =[re.sub("[(\n)(\r)]"," ",tweet) for tweet in df["text"]]
    entities = [tweet for tweet in df["entities"]]
    FC = [tweet for tweet in df["favorite_count"]]
    RT = [tweet for tweet in df["retweet_count"]]
    id_ = [tweet for tweet in df["id"]]
    j_user = [tweet for tweet in df["user"]]
    followers = [tweet["followers_count"] for tweet in output2["j_user"]]
    
    CA = df["created_at"]
    CA_l = [time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(CA[i],'%a %b %d %H:%M:%S +0000 %Y'))for i in range(0,len(CA))]
    
    output = pd.DataFrame([id_,Tweet,j_user,CA_l,entities,followers, FC,RT]).T
    output.reset_index(drop=True)
    output.columns = ["id", "Tweet", "j_user","created_at", "entities", "followers_user", "FC","RT"]
    return output

In [73]:
output2 = text_dataset(df)

In [76]:
pd.DataFrame.to_csv(output2, "C:/Users/Albert Prat/Documents/tweempact/datasets/tweet.csv")

In [1050]:
def trending_topic_df(df, path="c:/Users/Albert Prat/chromedriver.exe", just_dates=False):    
    from selenium import webdriver
    import pandas as pd
    import numpy as np
    import datetime
    
    if just_dates==False:
    # preparing elements to iterate
        textdf = df["Tweet"]
        cadf = df["created_at"]
        cadf = [date[:10] for date in cadf]
        cadf = np.unique(cadf)
        cadf = [date.split("-") for date in cadf]
        iterable_for_adding_dummy = [date[2]+date[1]+date[0] for date in cadf]
        # to be able to work with <, >
    if just_dates == True:
        iterable_for_adding_dummy = df
    # setting the minimum date to look for and preparing the filters   
    u = [datetime.datetime.strptime(date, "%d%m%Y") for date in iterable_for_adding_dummy]
    threshold = datetime.datetime.strptime("01032015", "%d%m%Y")
    filtr = [u[i]>=threshold for i in range(len(u))]
    filtr1 = [u[i]<threshold for i in range(len(u))]
    
    iterable = np.array(iterable_for_adding_dummy)[filtr]

    # which of the unique dates is before threshold?
    indices = [i for i, x in enumerate(filtr1) if x == True]
    
    # entering to browser and select the date box to be inputted
    browser = webdriver.Chrome(path)
    browser.get("https://trendogate.com/")
    date_dd = browser.find_element_by_xpath("//*[@id='userheaderbox']/div/form[2]/div[2]/input")
    
    # preparing output lists
    error = []
    grepl = []
    out = []
    for i in range(len(iterable)):
        browser.find_element_by_xpath("//select[@name='place']/option[text()='Spain']").click()
        date_dd = browser.find_element_by_xpath("//*[@id='userheaderbox']/div/form[2]/div[2]/input")
        date = uniq_dates[i][:10]
        date = date.split("-")
        date = date[2]+date[1]+date[0]
        date_dd.send_keys(date)
        browser.find_element_by_xpath("//*[@id='userheaderbox']/div/form[2]/button").click()

        if len(browser.find_elements_by_class_name("alert")) == 1:
            error.append(date)
            out.append(None)
            pass
        else:
            b = browser.find_element_by_class_name("col-lg-8")
            lgi = b.find_elements_by_class_name("list-group-item")
            TT = [el.text for el in lgi]
            TT = ",".join(TT)
            out.append(TT)
    for index in indices:
        out.insert(index, None)
    dates = [date[:2]+"/"+date[2:4]+"/"+date[4:] for date in iterable_for_adding_dummy]
    tocsv = pd.DataFrame([dates, out]).T
    tocsv.columns=["date", "trending_topics"]
    return tocsv

In [1051]:
tocsv = trending_topic_df(output2)

In [ ]:
def widen_TT_csv(df):
    old = pd.read_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\tweet.csv")
    ca_old = now["created_at"]
    ca_old = np.unique(ca_old)
    ca_old = [date.split("-") for date in ca_old]
    ca_old = [date[2]+date[1]+date[0] for date in ca_old]
    
    ca_new = df["created_at"]
    ca_new = np.unique(ca_new)
    ca_new = [date.split("-") for date in ca_new]
    ca_new = [date[2]+date[1]+date[0] for date in ca_new]
    
    new_dates = set(ca_new) - set(ca_old)
    tocsv = trending_topic_df(new_dates, just_dates = True)
    
    

In [1053]:
tocsv.to_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\trending_topics.csv")

In [1122]:
def append_dummy_TT(df):    
    textdf = df["Tweet"]
    trending_topics = pd.read_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\trending_topics.csv")
    TT = trending_topics["trending_topics"]
    indices = TT[TT.isnull()].index
    # get TT as a list of lists
    out = []
    for i in range(len(TT2)):
        if i in indices:
            out.append(None)
        else:
            t = TT2[i].split(",")
            out.append(t)
    # date management
    ca = list(df["created_at"])
    ca = [date[:10] for date in ca]
    sp_d = [date.split("-") for date in ca]
    sp_d = [date[2]+date[1]+date[0] for date in sp_d]
    # just keeping unique dates
    unique_date = np.unique(ca)
    unique_date = [date.split("-") for date in unique_date]
    unique_date = [date[2]+date[1]+date[0] for date in unique_date]
    # append sp_d for the sake of indexing
    df["spanish_date"]=sp_d
    grepl = []
    for i in range(len(sp_d)):
        ls = []
        date = df[df.index == i]["spanish_date"][i]
        tweet = textdf[i].lower()
        index = unique_date.index(date)
        TT = out[index]
        if TT == None:
            grepl.append(None)
        else:
            for word in TT:
                word = word.lower()
                ls.append(bool(re.search(r"\b"+str(word)+r"\b", tweet)))
            grepl.append(np.sum(ls))
    df = df.drop("spanish_date", axis = 1)
    df["trending_topic"]==grepl
    return df

In [1125]:
out2 = append_dummy_TT(output2)

In [1131]:
out2[out2["trending_topic"].isnull()]

,id,Tweet,j_user,created_at,entities,followers_user,FC,RT,trending_topic
1187,367540177868054528,@Design_it_fuer Campeonas 2013!!! Hemos conse...,"{'id': 1668653580, 'id_str': '1668653580', 'na...",2013-08-14 06:56:21,"{'hashtags': [], 'symbols': [], 'user_mentions...",2,1,1,NaN
2861,684599179016978433,@ANTH0NYCraft sube pirataland like para que a...,"{'id': 2889180230, 'id_str': '2889180230', 'na...",2016-01-06 04:55:38,"{'hashtags': [], 'symbols': [], 'user_mentions...",10,0,0,NaN
2862,683524946539094017,@rish_solano jajajjaja,"{'id': 2889180230, 'id_str': '2889180230', 'na...",2016-01-03 05:47:01,"{'hashtags': [], 'symbols': [], 'user_mentions...",10,0,0,NaN
2874,602502002522726400,@Manucrafting yo quiero ganar es la primera v...,"{'id': 2889180230, 'id_str': '2889180230', 'na...",2015-05-24 15:50:46,"{'hashtags': [], 'symbols': [], 'user_mentions...",10,0,0,NaN
2875,600443570433105920,@TownGamePlay puedes mandarle saludos a mi he...,"{'id': 2889180230, 'id_str': '2889180230', 'na...",2015-05-18 23:31:17,"{'hashtags': [], 'symbols': [], 'user_mentions...",10,0,0,NaN
2877,529446939478482945,@JIGAJAHITAKAVY perdón yora,"{'id': 2889180230, 'id_str': '2889180230', 'na...",2014-11-04 01:36:01,"{'hashtags': [], 'symbols': [], 'user_mentions...",10,1,0,NaN
2878,529446830678228993,@JIGAJAHITAKAVY de mi nessan jiji te quiero mu...,"{'id': 2889180230, 'id_str': '2889180230', 'na...",2014-11-04 01:35:35,"{'hashtags': [], 'symbols': [], 'user_mentions...",10,1,0,NaN


In [1134]:
out2.to_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\tweet.csv")

In [1135]:
pd.read_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\tweet.csv")

,Unnamed: 0,id,Tweet,j_user,created_at,entities,followers_user,FC,RT,trending_topic
0,0,1046429145914384386,Desde el #FCJumilla damos nuestra más sincera ...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 15:58:44,"{'hashtags': [{'text': 'FCJumilla', 'indices':...",7162,18,4,0.0
1,1,1044060298443919367,Me estoy volviendo loca hasta por cada una de ...,"{'id': 713359736, 'id_str': '713359736', 'name...",2018-09-24 03:05:47,"{'hashtags': [], 'symbols': [], 'user_mentions...",531,0,0,0.0
2,2,1046424001214648320,🏟 | El #FCJumilla ya está en La Condomina ⚽️...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 15:38:18,"{'hashtags': [{'text': 'FCJumilla', 'indices':...",7162,4,2,0.0
3,3,1046419267279818752,📹 | El vestuario del #FCJumilla en La Condomin...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 15:19:29,"{'hashtags': [{'text': 'FCJumilla', 'indices':...",7162,21,6,0.0
4,4,1046417005451128832,📑 Jornada 6 | El colegiado andaluz Guzmán Mans...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 15:10:30,"{'hashtags': [{'text': 'UCAMMurciaJumilla', 'i...",7162,0,0,0.0
5,5,1046416602739208193,🚐 | La expedición ya está lista para poner rum...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 15:08:54,"{'hashtags': [{'text': 'UCAMMurciaJumilla', 'i...",7162,10,2,0.0
6,6,1046343505197641728,⏱ | FINAL #FCJumillaB 1-1 @MulenoClubF ⚽️ ...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 10:18:26,"{'hashtags': [{'text': 'FCJumillaB', 'indices'...",7162,2,0,0.0
7,7,1046323550297239552,✋🏻⏱| DESCANSO #FCJumillaB 0-0 @MulenoClubF ...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 08:59:08,"{'hashtags': [{'text': 'FCJumillaB', 'indices'...",7162,0,0,0.0
8,8,1046300776455458816,🔊 VINÍCOLAS... 🔊 🔉 ¡¡El Filial también juega!...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 07:28:39,"{'hashtags': [], 'symbols': [], 'user_mentions...",7162,2,1,0.0
9,9,1045426450348888064,#CasadoDimisión,"{'id': 1401745850, 'id_str': '1401745850', 'na...",2018-09-27 21:34:23,"{'hashtags': [{'text': 'CasadoDimisión', 'indi...",329,0,0,0.0
